In [49]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

df = pd.read_csv("eplmatches.csv")
df.head()

,Season_End_Year,Wk,Date,Home,HomeGoals,AwayGoals,Away,FTR
0,1993,1,1992-08-15,Coventry City,2,1,Middlesbrough,H
1,1993,1,1992-08-15,Leeds United,2,1,Wimbledon,H
2,1993,1,1992-08-15,Sheffield Utd,2,1,Manchester Utd,H
3,1993,1,1992-08-15,Crystal Palace,3,3,Blackburn,D
4,1993,1,1992-08-15,Arsenal,2,4,Norwich City,A


In [77]:
def wins_by_year(team):
    df_team = df[((df["Home"] == team)&(df["FTR"] == 'H')) |
                ((df["Away"] == team)&(df["FTR"] == 'A'))]
    wins = df_team.value_counts("Season_End_Year").to_frame(team)
    wins['year'] = wins.index
    wins.sort_values(by='year', inplace=True)
    wins = wins.reset_index(drop=True)
    return wins

def stats_by_year(team):
    df_wins = df[((df["Home"] == team)&(df["FTR"] == 'H')) |
                ((df["Away"] == team)&(df["FTR"] == 'A'))]
    wins = df_wins.value_counts("Season_End_Year").to_frame("Wins")

    df_draws = df[((df["Home"] == team)&(df["FTR"] == 'D')) |
                ((df["Away"] == team)&(df["FTR"] == 'D'))]
    draws = df_draws.value_counts("Season_End_Year").to_frame("Draws")

    df_losses = df[((df["Home"] == team)&(df["FTR"] == 'A')) |
                ((df["Away"] == team)&(df["FTR"] == 'H'))]
    losses = df_losses.value_counts("Season_End_Year").to_frame("Losses")

    stats = wins.join(draws).join(losses)

    stats['year'] = stats.index
    stats.sort_values(by='year', inplace=True)
    stats.drop(['year'], axis=1, inplace=True)
    return stats


In [143]:
def get_color_team(team):
    team_colors = {
        "Manchester City": "Red"
    }
    if team in team_colors:
        return team_colors[team]
    return "MediumSeaGreen"

In [51]:
teams = df['Home'].unique()
wins_by_team = wins_by_year(teams[0]).set_index('year')
for team in teams[1:]:
    wins_by_team = wins_by_team.join(wins_by_year(team).set_index('year'), how='outer')

wins_by_team.head()

,Coventry City,Leeds United,Sheffield Utd,Crystal Palace,Arsenal,Ipswich Town,Everton,Southampton,Chelsea,Nott'ham Forest,...,Hull City,Stoke City,Burnley,Blackpool,Swansea City,Cardiff City,Bournemouth,Brighton,Huddersfield,Brentford
year,,,,,,,,,,,,,,,,,,,,,
1993,13.0,12.0,14.0,11.0,15,12.0,15,13.0,14,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1994,14.0,18.0,8.0,NaN,18,9.0,12,12.0,13,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1995,12.0,20.0,NaN,11.0,13,7.0,11,12.0,13,22.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996,8.0,12.0,NaN,NaN,17,NaN,17,9.0,12,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997,9.0,11.0,NaN,NaN,19,NaN,10,10.0,16,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [144]:
fav_team = 'QPR'
color_team = get_color_team(fav_team)

In [145]:
# creates subplots and defines favorite team
fig = make_subplots(rows=2, cols=1,
subplot_titles=("Vitórias Comparado aos Outros Times",
"Estatísticas Gerais"),
vertical_spacing=0.2)

# plots teams that are not the favorite team
for team in teams:
    if fav_team != team:
        fig.add_trace(go.Scatter(x=wins_by_team.index, y=wins_by_team[team],
         name=team,
         line=dict(color='gray', width=2),
         opacity=0.5,
         legendgroup="Outros times",
         legendgrouptitle_text="Outros times"),
         row=1, col=1)

# plots the favorite team
fig.add_trace(go.Scatter(x=wins_by_team.index, y=wins_by_team[fav_team],
             name=fav_team, 
             mode='lines+markers',
             marker=dict(color=color_team, size=10, line=dict(color='black', width=1.5)),
             line=dict(color=color_team, width=4),
             legendgroup=fav_team),
             row=1, col=1)

# plots the favorite team's stats
fig.add_trace(go.Bar(x=stats_by_year(fav_team).index, y=stats_by_year(fav_team)['Wins'],
                name='Vitórias',
                marker_color=color_team),
                row=2, col=1)
fig.add_trace(go.Bar(x=stats_by_year(fav_team).index, y=stats_by_year(fav_team)['Draws'],
                name='Empates',
                marker_color='SlateGray'),
                row=2, col=1)
fig.add_trace(go.Bar(x=stats_by_year(fav_team).index, y=stats_by_year(fav_team)['Losses'],
                name='Derrotas',
                marker_color='salmon'),
                row=2, col=1)
fig.update_layout(barmode='stack')


fig.update_layout(title=f'<b>{fav_team}</b> na Premier League por Ano',
                xaxis_title='Ano',
                yaxis_title='Vitórias',
                height=800,
                width=800)
fig.update_xaxes(matches='x')


fig.show()

In [146]:
# Plot pie chart with total stats for favorite team
from cProfile import label


fig = go.Figure(data=[go.Pie(labels=stats_by_year(fav_team).columns,
                                values=stats_by_year(fav_team).sum(),
                                hole=.3,
                                marker_colors=[color_team, 'SlateGray', 'salmon'])])
fig.update_layout(title=f'<b>{fav_team}</b> na Premier League no Geral',
                height=500,
                width=500)
fig.show()